# My hangman file

In [1]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
import pandas as pd
import numpy as np
import torch

In [2]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


Hmmmm, I should do a test-train split on the full dictionary...

In [3]:
from sklearn.model_selection import train_test_split
dict_train, dict_test = train_test_split(full_dict)

ughhhh how do I actually code up the AI model? I have the GAME, I need the PLAYER!


The model needs to accept:
an n-letter word, with letters/blanks encoded as one-hots (n by 27 dense vector)
a 1 x 26 vector of guesses made so far (0 for no guess, 1 for guess)

it needs to return as output:
1 x 26 vector of which argmax is the letter the model "chooses next"

In [4]:
n_epochs = 100
max_input_length = max([len(word) for word in dict_train])
batch_size = np.array([len(word) for word in dict_train]).mean()
print("Training set size: %i" % len(dict_train))
print("Max input size: %i" % max_input_length)
print("Average word length: %f" % batch_size)


Training set size: 170475
Max input size: 29
Average word length: 9.346514


In [5]:
max_input_length

29

In [6]:
len(full_dict)

227300

In [7]:
alphabet = list(string.ascii_lowercase)
orig_word = random.choice(dict_train)
blanked_word = "_" * len(orig_word)
#I want a distribution biased towards 0 instead of a flat one for choosing guesses
#do I though?
num_guesses = round(random.triangular(0,10))
guess_list = random.sample(alphabet, num_guesses)

if(num_guesses != 0):
    for guess_letter in guess_list:
        indices = [i for i, x in enumerate(list(orig_word)) if x == guess_letter]
        wordList = list(blanked_word)
        for i in indices:
            wordList[i] = guess_letter #now I replace the occurrences of the letter
        blanked_word = "".join(wordList)

remaining_correct_guesses = list(set(orig_word) - set(guess_list))

print(orig_word)
print(blanked_word)
print(guess_list)
print(remaining_correct_guesses)


layamon
__y____
['d', 'w', 'e', 'b', 'z', 'y']
['l', 'n', 'o', 'm', 'a']


Ok, so now what, we have input:

blanked word -> encode it
guess list -> encode it
remaining correct guesses *this is the target -> encode it?

how do I encode the blanked word though? the strings also have variable length...

Ok, convert word into numbers

In [12]:
longest_word_length = 45

In [27]:
def word_to_tensor(current_word):
    longest_word_length = 45 #this is the longest word in the english language
    #first, encode word: ex "apple"
    #assume word is something like "_pp__"
    alphabet = list(string.ascii_lowercase) 
    alphabet.append("_")#this is for blanks
    #alphabet.append("=") # using this for generic padding
    #input_length = longest_word_length + n_letters+1
    num_letters = len(alphabet)

    # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
    # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
    current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
    for widx, letter in enumerate(current_word):
        #print(widx)
        #print(letter)
        current_word_tensor[0][widx][alphabet.index(letter)] = 1
    #print(current_word_tensor)
    return current_word_tensor

def guess_list_to_tensor(guess_list):
    #need to encode the guess list as a 1 x 26 tensor
    guess_tensor = torch.zeros(1, 26)
    for letter in guess_list:
        guess_tensor[0][alphabet.index(letter)] = 1
    
    return guess_tensor

    

In [13]:
print(word_to_tensor("_pp__"))
print(guess_list_to_tensor(["p"]))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0.]]])


In [11]:
print(word_to_tensor("_pp__").shape)
print(guess_list_to_tensor(["p"]).shape)

torch.Size([1, 27, 45])
torch.Size([1, 1, 26])


Ok, so the current word is converted into a tensor and then fed into a LSTM, the guess list is converted to a tensor and then COMBINED with the output from the LSTM and then that combo is fed into a dense layer.

In [216]:
import torch.nn as nn

class Player(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Player, self).__init__()
        self.num_classes = num_classes #number of outputs
        self.num_layers = num_layers #I think this is arbitrary
        self.input_size =  input_size #input features
        self.hidden_size = hidden_size #to match number of letters in encoded guess
        self.seq_length = seq_length #to match number of letters in input
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, 26) #fully connected last layer
        self.softmax = nn.LogSoftmax(dim=1)


    
    def forward(self, encoded_word, encoded_guess):
        h_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #hidden state
        c_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        #self.lstm.flatten_parameters()
        out, (hn, cn) = self.lstm(encoded_word, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = out[:,-1]
        #print(out.shape)
        #print('The shape of lstm_out is:', out.shape)
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        #print(hn.shape)
        #print(encoded_guess.shape)
        hn = torch.cat((hn,encoded_guess)) #combine data with encoded guess data before passing to dense layer



        out = self.fc_1(out) #first Dense
        #print(out.shape)

        out = self.fc(out) #Final Output
        #print(out.shape)
        out = self.softmax(out)
        return out
    



    

In [188]:
longest_word_length = 45
player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
#print(player1)
current_word = "_pp__"
guess_list = ["p"]
word_tensor = word_to_tensor(current_word)
guess_tensor = guess_list_to_tensor(guess_list)

player_output = player1(word_tensor, guess_tensor)

torch.Size([1, 26])


In [180]:
t1 = player_output.detach().numpy()[0]

In [182]:
t1

array([0.04413354, 0.03679811, 0.0347988 , 0.04169655, 0.03848374,
       0.03703758, 0.04089747, 0.03275016, 0.04235902, 0.03814205,
       0.03962979, 0.0428115 , 0.03940127, 0.03494367, 0.04051181,
       0.04107698, 0.04093184, 0.04125075, 0.03831664, 0.03834148,
       0.03236494, 0.03525997, 0.04113822, 0.03843417, 0.03242484,
       0.03606509], dtype=float32)

In [199]:
torch.argmax(player_output[0])

tensor(16)

In [200]:
def guess_from_output(output, guess_list):
    #output is the tensor I get from player
    output = output.detach().numpy()[0] #this is just easier to deal with

    alphabet = list(string.ascii_lowercase)
    idx = np.argmax(output)
    print(idx)
    guess = alphabet[idx]
    #I will here force the guess to be a new one...
    while guess in guess_list:
        output[idx] -= 1 
        idx = np.argmax(output)
        guess = alphabet[idx]


    return guess

In [236]:
def error_propagate(output, answer):
    #can use the guess_list_to_tensor to convert the answer word into a tensor
    answer_tensor = guess_list_to_tensor(list(answer))
    answer_tensor = answer_tensor.long()
    print(answer_tensor)
    loss = nn.NLLLoss()
    error = loss(torch.flatten(output), torch.flatten(answer_tensor))
    error.backward()    

In [237]:
error_propagate(player_output, "apple")

tensor([[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])


In [202]:
guess_list_to_tensor(list("apple"))

tensor([[1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])

In [198]:
print(guess_from_output(player_output, guess_list))

16
q


Can almost start learning...


before start,
Initialize player object
Initialize hangman object

Each loop of training will:

generate correct word and set up game
zero out initial state of player



In [256]:
class MyHangman4(object):
    def __init__(self, player, dictionary):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        #full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = dictionary #going to use a subset of the training data obviously
        #self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        self.has_won = False
        self.player = player
        self.learning_rate = 0.005 #should have this as an input?
        self.current_loss = 0
        
    #def generate_correct_word(self):
    #    self.correct_word = random.choice(self.full_dictionary)

    def get_winstate(self):
        return self.has_won
    
    #def add_player(self,player):
    #    self.player = player

    def word_to_tensor(current_word):
        longest_word_length = 45 #this is the longest word in the english language
        #first, encode word: ex "apple"
        #assume word is something like "_pp__"
        alphabet = list(string.ascii_lowercase) 
        alphabet.append("_")#this is for blanks
        #alphabet.append("=") # using this for generic padding
        #input_length = longest_word_length + n_letters+1
        num_letters = len(alphabet)

        # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
        # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
        current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
        for widx, letter in enumerate(current_word):
            #print(widx)
            #print(letter)
            current_word_tensor[0][widx][alphabet.index(letter)] = 1
        #print(current_word_tensor)
        return current_word_tensor

    def guess_list_to_tensor(guess_list):
        #need to encode the guess list as a 1 x 26 tensor
        guess_tensor = torch.zeros(1, 26)
        for letter in guess_list:
            guess_tensor[0][alphabet.index(letter)] = 1
        
        return guess_tensor


    def guess_from_output(self,output, guess_list):
        #output is the tensor I get from player
        output = output.detach().numpy()[0] #this is just easier to deal with

        alphabet = list(string.ascii_lowercase)
        idx = np.argmax(output)
        #print(idx)
        guess = alphabet[idx]
        #I will here force the guess to be a new one...
        while guess in guess_list:
            output[idx] -= 1 
            idx = np.argmax(output)
            guess = alphabet[idx]
        return guess
    
    def error_propagate(self, output, answer):
        #can use the guess_list_to_tensor to convert the answer word into a tensor
        answer_tensor = self.guess_list_to_tensor(list(answer))
        answer_tensor = answer_tensor.long()
        loss = nn.NLLLoss()
        #print(output.shape)
        #print(answer_tensor.shape)
        error = loss(torch.flatten(output), torch.flatten(answer_tensor))
        error.backward()    
        #record stuff so we can plot later
        for p in self.player.parameters():
            p.data.add(p.grad.data, alpha=-self.learning_rate)

        self.current_loss = error
        

    #everything gets put together in the guess function
    def guess(self, player, word, guess_list):
        word_tensor = word_to_tensor(word)
        guess_list_tensor = guess_list_to_tensor(guess_list)
        output = player(word_tensor, guess_list_tensor)
        self.error_propagate(output, self.correct_word)
        guess_letter = self.guess_from_output(output, guess_list)

        return guess_letter
    
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)

    def get_loss(self):
        return self.current_loss

    

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        self.has_won = False
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?

        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer
            self.player.zero_grad()

            #get guessed letter
            guess_letter = self.guess(self.player, word, self.guessed_letters)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                

In [257]:

learning_rate = 0.005
player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
game = MyHangman4(player1, dict_train)
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _____________.
Guessing letter: h
Got a Letter, h, # of tries remaining: 6. Word: ____________h.
Guessing letter: b
Incorrect guess b, # of tries remaining: 6. Word: ____________h.
Guessing letter: k
Incorrect guess k, # of tries remaining: 5. Word: ____________h.
Guessing letter: s
Got a Letter, s, # of tries remaining: 4. Word: _______s___sh.
Guessing letter: t
Got a Letter, t, # of tries remaining: 4. Word: ___t___s___sh.
Guessing letter: m
Incorrect guess m, # of tries remaining: 4. Word: ___t___s___sh.
Guessing letter: z
Incorrect guess z, # of tries remaining: 3. Word: ___t___s___sh.
Guessing letter: x
Incorrect guess x, # of tries remaining: 2. Word: ___t___s___sh.
Guessing letter: v
Incorrect guess v, # of tries remaining: 1. Word: ___t___s___sh.
You Lose, the answer was: picturesquish


In [260]:
n_iters = 1000
print_every = 50

player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
game = MyHangman4(player1, dict_train)

error_list = []
t = time.time()
elapsed = time.time() - t
wincount = 0

for i in range(1, n_iters+1):
    game.start_game(verbose = False)
    error_list.append(game.get_loss())
    wincount += game.get_winstate()

    if(i % print_every == 0):
        elapsed = time.time() - t
        print("On Iteration %i of %i" % (i, n_iters))
        print("Current loss is: %f" % game.get_loss())
        print("Has been running for: %f seconds" % elapsed)
    

On Iteration 50 of 1000
Current loss is: 3.333233
Has been running for: 2.858845 seconds
On Iteration 100 of 1000
Current loss is: 3.388960
Has been running for: 5.554532 seconds
On Iteration 150 of 1000
Current loss is: 3.333233
Has been running for: 8.296739 seconds
On Iteration 200 of 1000
Current loss is: 3.333233
Has been running for: 11.049318 seconds
On Iteration 250 of 1000
Current loss is: 3.388960
Has been running for: 13.725932 seconds
On Iteration 300 of 1000
Current loss is: 3.388960
Has been running for: 16.406049 seconds
On Iteration 350 of 1000
Current loss is: 3.333233
Has been running for: 19.083381 seconds
On Iteration 400 of 1000
Current loss is: 3.388960
Has been running for: 21.747349 seconds
On Iteration 450 of 1000
Current loss is: 3.333233
Has been running for: 24.600852 seconds
On Iteration 500 of 1000
Current loss is: 3.333233
Has been running for: 27.320525 seconds
On Iteration 550 of 1000
Current loss is: 3.388960
Has been running for: 29.880100 seconds
On 

In [263]:
game.player.parameters

<bound method Module.parameters of Player(
  (lstm): LSTM(45, 26, num_layers=10, batch_first=True)
  (fc_1): Linear(in_features=26, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=26, bias=True)
  (softmax): LogSoftmax(dim=1)
)>

In [261]:
wincount

0

In [ ]:
n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
game = MyHangman4(player1, dict_train)

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print ``iter`` number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0





In [47]:
game.get_winstate()

True

In [10]:
for i in range(10):
    game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Incorrect guess s, # of tries remaining: 5. Word: _________.
Guessing letter: o
Got a Letter, o, # of tries remaining: 4. Word: _o_______.
Guessing letter: t
Got a Letter, t, # of tries remaining: 4. Word: to_______.
Guessing letter: r
Got a Letter, r, # of tries remaining: 4. Word: torr_____.
Guessing letter: i
Got a Letter, i, # of tries remaining: 4. Word: torr__i__.
Guessing letter: e
Got a Letter, e, # of tries remaining: 4. Word: torre_ie_.
Guessing letter: d
Got a Letter, d, # of tries remaining: 4. Word: torre_ied.
Guessing letter: f
Success! the word was: torrefied
Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: n
Incorrect guess n, # of tries remaining: 6. Word: _________.
Guessing letter: s
Got a Letter, s, # of tries remaining: 5. Word: ______s_s.
Guessing letter:

In [2]:
full_dict

NameError: name 'full_dict' is not defined